In [ ]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import imageio
import xml.etree.ElementTree as ET

import czifile
import tifffile as tiff

from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, binary_erosion
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from scipy.ndimage import binary_fill_holes
import napari

# NEW imports for 3D textured surface
from scipy.ndimage import map_coordinates, gaussian_filter as ndi_gaussian_filter
from skimage.measure import marching_cubes

# ===============================
# USER CONFIGURATION
# ===============================
# Path to your file (.tif or .czi)
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.tif"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.tif"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/TMEM-HA 6h CB airy_3_170722.tif"

# Correct microscope calibration (µm per pixel)
DEFAULT_VX_VY_UM = 0.04   # <-- YOUR TRUE XY SCALE
MAX_REASONABLE_VXY_UM = 0.5  # µm/px sanity limit

# ---------------------------
# Helper: refine radii by distance transform
# ---------------------------
def refine_radii_via_dt(img3d, blobs, win_px=25, bin_method="sauvola"):
    from skimage.filters import threshold_sauvola, threshold_local, threshold_otsu
    from skimage.morphology import remove_small_objects, disk, binary_opening
    from scipy.ndimage import distance_transform_edt as _edt
    from skimage.measure import label as _label

    if blobs is None or len(blobs) == 0:
        return blobs

    Z, H, W = img3d.shape
    out = blobs.copy().astype(np.float32)

    for i, (zc, yc, xc, _) in enumerate(out):
        z, y, x = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= z < Z and 0 <= y < H and 0 <= x < W):
            continue

        y1, y2 = max(0, y - win_px), min(H, y + win_px + 1)
        x1, x2 = max(0, x - win_px), min(W, x + win_px + 1)
        patch = img3d[z, y1:y2, x1:x2]

        if bin_method == "sauvola":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_sauvola(patch, window_size=ws, k=0.2)
            bw = patch > thr
        elif bin_method == "local":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_local(patch, block_size=ws, offset=-0.2*np.std(patch))
            bw = patch > thr
        else:
            try:
                thr = threshold_otsu(patch)
                bw = patch > thr
            except ValueError:
                continue

        bw = binary_opening(bw, footprint=disk(1))
        bw = remove_small_objects(bw, min_size=3, connectivity=2)

        yy, xx = y - y1, x - x1
        if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]) or not bw[yy, xx]:
            continue

        lab = _label(bw)
        lbl = lab[yy, xx]
        if lbl == 0:
            continue
        bw_obj = (lab == lbl)
        dt = _edt(bw_obj)
        r_px = float(dt[yy, xx])
        if r_px <= 0:
            continue
        out[i, 3] = r_px

    return out

# ===============================
# Image loading and metadata
# ===============================
def _try_float(x):
    try:
        return float(x)
    except:
        return None

def _parse_ome_xml(xml_text):
    if not xml_text:
        return None
    def grab(attr):
        m = re.search(fr'PhysicalSize{attr}="([\d\.eE+-]+)"', xml_text)
        return float(m.group(1)) if m else None
    return grab("X"), grab("Y"), grab("Z")

def load_any(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext in (".tif", ".tiff"):
        with tiff.TiffFile(file_path) as tf:
            arr = tf.asarray()
            try:
                ome_xml = tf.ome_metadata
            except Exception:
                ome_xml = None
            vx_um = vy_um = vz_um = None
            if ome_xml:
                vx_um, vy_um, vz_um = _parse_ome_xml(ome_xml)
        img = np.squeeze(arr)
        if img.ndim == 4:
            if img.shape[0] == 2:        # (C, Z, Y, X)
                ch1, ch2 = img[0], img[1]
            elif img.shape[1] == 2:      # (Z, C, Y, X)
                ch1, ch2 = img[:, 0], img[:, 1]
            elif img.shape[-1] == 2:     # (Z, Y, X, C)
                ch1, ch2 = img[..., 0], img[..., 1]
            else:
                raise RuntimeError("Unexpected TIFF shape for 2 channels")
        else:
            raise RuntimeError("Unexpected TIFF shape")
        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "tiff"}
    else:
        raise ValueError("Unsupported file format")

# ==========================================
# Load image and scales
# ==========================================
img_ch1, img_ch2, (vx_um, vy_um, vz_um), meta = load_any(file_path)
if vx_um is None or vy_um is None:
    vx_um = vy_um = DEFAULT_VX_VY_UM
if vz_um is None:
    vz_um = vx_um

if vx_um > MAX_REASONABLE_VXY_UM:
    raise ValueError(f"XY pixel size too large: {vx_um} µm/px")

px_um_xy = float(np.sqrt(vx_um * vy_um))
voxel_um3 = vx_um * vy_um * vz_um
print(f"Voxel size (µm): X={vx_um}, Y={vy_um}, Z={vz_um}")

# ===== Aliases used throughout =====
image   = img_ch1       # lysosome channel (Ch1)
image_2 = img_ch2       # neuron / context channel (Ch2)

# ==========================================
# Lysosome detection and scaling (kept as-is)
# ==========================================
image_smooth = gaussian(image, sigma=1.5)

blobs = blob_log(image_smooth, min_sigma=0.8, max_sigma=3.0, num_sigma=4, threshold=0.003)
if len(blobs) > 0:
    blobs[:, 3] *= np.sqrt(3)  # LoG radius correction (pixels)

blobs = refine_radii_via_dt(image_smooth, blobs)  # DT-refined radius (pixels)

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um = blobs[:, 3] * px_um_xy
    diameter_um = 2 * radius_um
    volume_um3 = (4/3) * np.pi * radius_um**3
    blob_ids = np.arange(1, len(blobs)+1, dtype=int)
else:
    z_um = y_um = x_um = radius_um = diameter_um = volume_um3 = np.array([])
    blob_ids = np.array([], dtype=int)

df = pd.DataFrame({
    "id": blob_ids,
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "radius_um": radius_um,
    "diameter_um": diameter_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv")

# === NEW: create a second dataset with ALL distinct radius_um values (±5%) and clamp to [0.0, 0.4] µm ===
def _unique_radii_within_5pct(radius_series, low=0.0, high=0.4, max_frac=0.05):
    arr = radius_series.to_numpy().astype(float)
    uniq = np.empty_like(arr, dtype=float)
    used = set()
    for i, r in enumerate(arr):
        base = float(np.clip(r, low, high))
        dev = max(max_frac * max(abs(base), 1e-6), 1e-9)
        if base not in used:
            uniq[i] = base
            used.add(base)
            continue
        found = False
        step = dev / 20.0
        for k in range(1, 401):
            sgn = 1.0 if (k % 2 == 1) else -1.0
            cand = float(np.clip(base + sgn * min(dev, k * step), low, high))
            if abs(cand - base) <= dev and cand not in used:
                uniq[i] = cand
                used.add(cand)
                found = True
                break
        if not found:
            cand = float(np.clip(np.nextafter(base, high), low, high))
            uniq[i] = cand
            used.add(cand)
    return pd.Series(uniq, index=radius_series.index, name="radius_um")

if len(df) > 0:
    df_unique = df.copy()
    df_unique["radius_um"] = _unique_radii_within_5pct(df_unique["radius_um"], low=0.0, high=0.4, max_frac=0.05)
    df_unique["diameter_um"] = 2.0 * df_unique["radius_um"]
    df_unique["volume_um3"]  = (4.0/3.0) * np.pi * (df_unique["radius_um"] ** 3)
    df_unique.to_csv("lysosome_blobs_regions_unique_radius.csv", index=False)
    print("Saved: lysosome_blobs_regions_unique_radius.csv (all radii distinct, ±5% max change, clamped to [0.0, 0.4] µm)")
    df = df_unique.copy()

# ==========================================
# CH2 segmentation (CELL vs OUTSIDE)
# ==========================================
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=1.5, preserve_range=True)

# Local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=301, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=20000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(10))
neuron_mask = binary_erosion(neuron_mask, ball(7))

# Soma via distance + cleanup
dist = edt(neuron_mask)
cell_min_radius_vox = 1
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(5))
cell_mask = binary_closing(cell_mask, ball(10))
cell_mask = binary_fill_holes(cell_mask)

# Label and territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# ==========================================
# Map lysosomes to (cell/outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list = []

if len(df) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in df[["z_um","y_um","x_um"]].to_numpy() / np.array([vz_um, vy_um, vx_um]):
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue
        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# Include zero-count cells
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
        all_cells = pd.DataFrame({
            "cell_id_ch2": np.arange(1, int(cell_seg.max()) + 1, dtype=int)
        })

        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts_nonzero = (
                df[df["location_ch2"] == "cell"]
                .groupby("cell_id_ch2")
                .size()
                .reset_index(name="count")
            )
        else:
            lys_counts_nonzero = pd.DataFrame(columns=["cell_id_ch2", "count"])

        lys_counts_all = (
            all_cells.merge(lys_counts_nonzero, on="cell_id_ch2", how="left")
                     .fillna({"count": 0})
        )
        lys_counts_all["count"] = lys_counts_all["count"].astype(int)

        lys_counts_all.to_csv("lysosome_counts_by_cell.csv", index=False)
        print("Updated: lysosome_counts_by_cell.csv now includes cells with 0 lysosomes.")
except Exception as e:
    print("Could not expand lysosome_counts_by_cell with zero-count cells:", e)

# ==========================================
# Per-cell (Ch2) volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# Napari visualization layers (add AFTER segmentation exists)
# ==========================================
viewer = napari.Viewer()

# --- Hover-only labels helper (ADDED) ---
def attach_hover_tooltip(points_layer, viewer, template,
                         color='white', size=10, anchor='upper_left'):
    """
    Show a label only for the point under the cursor.
    `template` can use properties keys, e.g. 'C:{cell}  ID:{lys_id}  d:{diameter_um:.3f}µm'
    """
    import numpy as _np
    n = len(points_layer.data)
    strings = _np.array([''] * n, dtype=object)
    points_layer.text = {'string': strings, 'color': color, 'size': size, 'anchor': anchor}
    state = {'last': None}

    def _on_move(layer, event):
        # get index of point under cursor (or None)
        try:
            idx = layer.get_value(
                event.position,
                view_direction=viewer.camera.view_direction,
                dims_displayed=viewer.dims.displayed,
                world=True
            )
        except TypeError:
            idx = layer.get_value(
                event.position,
                view_direction=viewer.camera.view_direction,
                dims_displayed=viewer.dims.displayed
            )
        if isinstance(idx, (list, tuple)):
            idx = idx[0] if idx else None

        if idx == state['last']:
            return

        if state['last'] is not None and 0 <= state['last'] < n:
            strings[state['last']] = ''

        if idx is not None and 0 <= idx < n:
            props = {k: v[idx] for k, v in points_layer.properties.items()}
            try:
                strings[idx] = template.format(**props)
            except Exception:
                strings[idx] = ''

        state['last'] = idx
        points_layer.text = {'string': strings, 'color': color, 'size': size, 'anchor': anchor}

    points_layer.mouse_move_callbacks.append(_on_move)

# base images
viewer.add_image(img_ch2, name="Ch2 raw")
viewer.add_image(img_ch1, name="Ch1 raw")

# cell mask and segmentation layers
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'

    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosome points overlay (show ONLY lysosomes inside cells) + HOVER tooltips (2D)
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    in_cell_mask = (df["location_ch2"].to_numpy() == "cell")
    if in_cell_mask.any():
        blobs_cell = np.stack([
            df.loc[in_cell_mask, "z_um"].to_numpy() / vz_um,
            df.loc[in_cell_mask, "y_um"].to_numpy() / vy_um,
            df.loc[in_cell_mask, "x_um"].to_numpy() / vx_um
        ], axis=1)
        radii_vox = (df.loc[in_cell_mask, "radius_um"].to_numpy() / (np.sqrt(vx_um * vy_um) + 1e-12))
        pts = viewer.add_points(
            blobs_cell,
            size=np.clip(radii_vox * 2, 2, None),
            name='Lysosomes (cell only)'
        )
        try:
            pts.face_color = [0.0, 1.0, 1.0, 1.0]       # cyan
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df.loc[in_cell_mask, 'id'].to_numpy(),
                'cell':   df.loc[in_cell_mask, 'cell_id_ch2'].to_numpy(),
                'diameter_um': df.loc[in_cell_mask, 'diameter_um'].to_numpy()
            }
            # Hover-only label (CHANGE)
            attach_hover_tooltip(
                pts, viewer,
                template='C:{cell}  ID:{lys_id}  d:{diameter_um:.3f}µm',
                color='yellow', size=10, anchor='upper_left'
            )
        except Exception:
            pass

# ===== NEW LAYER: show ALL lysosomes (inside + outside) + HOVER tooltips =====
if len(df) > 0:
    try:
        blobs_all = np.stack([
            (df["z_um"].to_numpy() / vz_um),
            (df["y_um"].to_numpy() / vy_um),
            (df["x_um"].to_numpy() / vx_um)
        ], axis=1)
        radii_all_vox = df["radius_um"].to_numpy() / (np.sqrt(vx_um * vy_um) + 1e-12)
        pts_all = viewer.add_points(
            blobs_all,
            size=np.clip(radii_all_vox * 2, 2, None),
            name='Lysosomes (all)'
        )
        pts_all.face_color = [1.0, 1.0, 1.0, 1.0]  # white
        pts_all.edge_color = 'black'
        pts_all.edge_width = 0.3
        pts_all.properties = {
            'lys_id': df['id'].to_numpy(),
            'cell': df.get('cell_id_ch2', pd.Series(np.zeros(len(df), dtype=int))).to_numpy(),
            'where': df.get('location_ch2', pd.Series(['unknown']*len(df))).to_numpy(),
            'diameter_um': df['diameter_um'].to_numpy()
        }
        # Hover-only label (CHANGE)
        attach_hover_tooltip(
            pts_all, viewer,
            template='C:{cell}  ID:{lys_id}  d:{diameter_um:.3f}µm  {where}',
            color='white', size=9, anchor='upper_left'
        )
    except Exception:
        pass

# ==========================================
# 3D textured surface of the cell + lysosomes in 3D
# ==========================================
def add_textured_cell_surface(viewer, cell_mask, texture_vol,
                              iso_level=0.5, step_size=2,
                              smooth_sigma_vox=1.0,
                              opacity=0.75, colormap='turbo'):
    """
    Builds a surface from cell_mask and uses texture_vol intensities as per-vertex colors.
    spacing=(vz_um, vy_um, vx_um) keeps the mesh in real µm units.
    """
    # smooth the binary mask a bit for a cleaner surface
    mask_f = cell_mask.astype(np.float32)
    if smooth_sigma_vox and smooth_sigma_vox > 0:
        mask_f = ndi_gaussian_filter(mask_f, smooth_sigma_vox)

    verts, faces, normals, vals = marching_cubes(
        volume=mask_f,
        level=iso_level,
        spacing=(vz_um, vy_um, vx_um),
        step_size=step_size,
        allow_degenerate=False
    )

    z_vox = verts[:, 0] / vz_um
    y_vox = verts[:, 1] / vy_um
    x_vox = verts[:, 2] / vx_um
    tex = map_coordinates(texture_vol.astype(np.float32), [z_vox, y_vox, x_vox],
                          order=1, mode='nearest')

    tmin, tmax = float(tex.min()), float(tex.max())
    if tmax > tmin:
        tex = (tex - tmin) / (tmax - tmin)
    else:
        tex = np.zeros_like(tex, dtype=np.float32)

    surf = viewer.add_surface((verts, faces, tex), name='Cell surface (textured)')
    surf.colormap = colormap
    surf.shading  = 'smooth'
    surf.opacity  = opacity
    return surf

# ensure 3D
viewer.dims.ndisplay = 3

# soften Ch2 for nicer texture and add the textured surface
ch2_tex = gaussian(image_2.astype(np.float32), sigma=1.0, preserve_range=True)
_ = add_textured_cell_surface(
    viewer,
    cell_mask=cell_mask,
    texture_vol=ch2_tex,
    iso_level=0.5,
    step_size=2,
    smooth_sigma_vox=1.0,
    opacity=0.75,
    colormap='turbo'
)

# emphasize lysosomes INSIDE the cell in 3D (explicit 3D layer) + HOVER tooltips
if len(df) > 0:
    in_mask  = (df["location_ch2"].to_numpy() == "cell")
    if in_mask.any():
        pts_zyx_um = np.stack([
            df.loc[in_mask, "z_um"].to_numpy(),
            df.loc[in_mask, "y_um"].to_numpy(),
            df.loc[in_mask, "x_um"].to_numpy()
        ], axis=1)

        radii_vox = (df.loc[in_mask, "radius_um"].to_numpy() / (np.sqrt(vx_um * vy_um) + 1e-12))
        sizes_3d = np.clip(radii_vox * 2.5, 2.0, None)

        pts3d = viewer.add_points(
            np.stack([pts_zyx_um[:, 0] / vz_um,
                      pts_zyx_um[:, 1] / vy_um,
                      pts_zyx_um[:, 2] / vx_um], axis=1),
            name='Lysosomes (inside, 3D)',
            size=sizes_3d
        )
        try:
            pts3d.face_color = [1.0, 1.0, 0.0, 1.0]   # bright yellow
            pts3d.edge_color = 'black'
            pts3d.edge_width = 0.2
        except Exception:
            pass

        pts3d.properties = {
            'cell_id':     df.loc[in_mask, 'cell_id_ch2'].to_numpy(),
            'lys_id':      df.loc[in_mask, 'id'].to_numpy(),
            'diameter_um': df.loc[in_mask, 'diameter_um'].to_numpy()
        }
        # Hover-only label (CHANGE)
        attach_hover_tooltip(
            pts3d, viewer,
            template='C:{cell_id}  ID:{lys_id}  d:{diameter_um:.3f}µm',
            color='white', size=11, anchor='upper_left'
        )

# optional: set a nice camera pose
try:
    viewer.camera.zoom = 1.2
except Exception:
    pass

# ==========================================
# Optional fused 2D video (detect ALL lysosomes: inside + outside)
# ==========================================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]

for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    drew_any = False

    # --- Preferred: draw from df (both inside and outside)
    if 'df' in locals() and isinstance(df, pd.DataFrame) and len(df) > 0:
        # indices in this z-slice
        z_vox_all = np.clip(np.rint(df["z_um"].to_numpy() / vz_um).astype(int), 0, Z - 1)
        sel = (z_vox_all == z)
        if np.any(sel):
            ys = np.rint(df.loc[sel, "y_um"].to_numpy() / vy_um).astype(int)
            xs = np.rint(df.loc[sel, "x_um"].to_numpy() / vx_um).astype(int)
            r_vox = (df.loc[sel, "radius_um"].to_numpy() /
                     (np.sqrt(vx_um * vy_um) + 1e-12))
            for y, x, r in zip(ys, xs, r_vox):
                rr = int(max(2, round(r)))
                if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2]:
                    # draw ALL lysosomes regardless of cell membership
                    cv2.circle(overlay, (x, y), rr, (255, 255, 0), 2)  # yellow
            drew_any = True

    # --- Fallback: use LoG detections if df isn't available
    if (not drew_any) and ('blobs' in locals()) and (blobs is not None) and (len(blobs) > 0):
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2]:
                # draw ALL lysosomes (no cell_mask filter)
                cv2.circle(overlay, (x, y), r, (255, 255, 0), 2)  # yellow

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# Optional fused 2D video (RAW uses ALL image: Ch1+Ch2) with perfect sync
# ==========================================

def _norm_u8_stack(vol):
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        out = (np.clip((vol - vmin) / (vmax - vmin), 0, 1) * 255.0).astype(np.uint8)
    else:
        out = np.zeros_like(vol, dtype=np.uint8)
    return out

# Per-volume normalization to keep brightness stable across z
ch1_u8 = _norm_u8_stack(img_ch1.astype(np.float32))
ch2_u8 = _norm_u8_stack(ch2.astype(np.float32) * 255.0 / max(1.0, ch2.max()))  # ch2 already 0..1

frames_raw = []
frames_fused = []
frames_side_by_side = []

Z = ch2_u8.shape[0]

# Precompute per-z lists of in-cell lysosomes from df
in_cell_per_z = [[] for _ in range(Z)]
if len(df) > 0 and "location_ch2" in df.columns:
    in_df = df[df["location_ch2"].to_numpy() == "cell"]
    if len(in_df) > 0:
        z_vox = np.clip(np.rint(in_df["z_um"].to_numpy() / vz_um).astype(int), 0, Z - 1)
        y_vox = np.rint(in_df["y_um"].to_numpy() / vy_um).astype(int)
        x_vox = np.rint(in_df["x_um"].to_numpy() / vx_um).astype(int)
        r_vox = np.clip(
            (in_df["radius_um"].to_numpy() / (np.sqrt(vx_um * vy_um) + 1e-12)),
            1.0, None
        )
        for zz, yy, xx, rr in zip(z_vox, y_vox, x_vox, r_vox):
            in_cell_per_z[zz].append((yy, xx, int(max(2, round(rr)))))

for z in range(Z):
    # RAW frame (both channels): BGR composition (Ch1->magenta, Ch2->green)
    b = ch1_u8[z]
    g = ch2_u8[z]
    r = ch1_u8[z]
    base = np.dstack([b, g, r])

    # FUSED overlay (green cell mask + in-cell lysosomes)
    cell = (cell_mask[z].astype(np.uint8) * 255)
    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(in_cell_per_z[z]) > 0:
        for (y, x, rpx) in in_cell_per_z[z]:
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                cv2.circle(overlay, (x, y), rpx, (255, 255, 0), 2)
    else:
        if 'blobs' in locals() and len(blobs) > 0:
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b_ in z_blobs:
                y, x = int(round(b_[1])), int(round(b_[2]))
                rpx = max(2, int(round(b_[3])))
                if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                    cv2.circle(overlay, (x, y), rpx, (255, 255, 0), 2)

    cv2.putText(base,    "RAW (Ch1+Ch2)",        (10, 22), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(overlay, "FUSED (in-cell only)", (10, 22), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2, cv2.LINE_AA)

    frames_raw.append(base)
    frames_fused.append(overlay)
    frames_side_by_side.append(cv2.hconcat([base, overlay]))

# --- Consistent save with identical timing for all videos ---
FPS = 8

def _save_video_sync(basename, frames):
    try:
        with imageio.get_writer(f"{basename}.mp4",
                                fps=FPS,
                                format="FFMPEG",
                                codec="libx264",
                                macro_block_size=None) as w:
            for fr in frames:
                w.append_data(fr)
        print(f"Saved: {basename}.mp4 @ {FPS} fps")
    except Exception:
        imageio.mimsave(f"{basename}.gif", frames, duration=1.0/FPS)
        print(f"Saved: {basename}.gif @ {FPS} fps equivalent")

_save_video_sync('ch2_fused_cell', frames_fused)
_save_video_sync('ch2_raw', frames_raw)
_save_video_sync('ch2_raw_and_fused', frames_side_by_side)

# ==========================================
# Run viewer
# ==========================================
napari.run()